# Sample Notebook
This is a sample Jupyter Notebook uploaded to GitHub.

In [2]:
# 1. GPU check
!nvidia-smi

# 2. Install dependencies
!pip -q install -U transformers datasets sentencepiece accelerate huggingface_hub python-dotenv feedparser beautifulsoup4 sacrebleu rouge-score gradio



# 3. Clone or pull repo
from getpass import getpass
token = getpass("Enter your GitHub PAT: ")
USERNAME = "sukritichawla"
REPO = "legaldocs_ST"
REPO_URL = f"https://{token}@github.com/{USERNAME}/{REPO}.git"
!git clone $REPO_URL


# 4. Go inside repo
%cd /content/legaldocs_ST

# 5. Hugging Face login
from huggingface_hub import loginhf_XvxVrdjfbWaWTUJvdGzRUphKRYrUeWxrRr
login()

Fri Sep 12 16:34:40 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   55C    P8             12W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [12]:
from transformers import pipeline

# Choose a simplification fine-tuned model (change to any HF model above)
model_name = "agentlans/flan-t5-small-simplifier"  # or "mrm8488/t5-small-finetuned-text-simplification"

simplifier = pipeline("text2text-generation", model=model_name, tokenizer=model_name, device=-1)  # device=0 for GPU

legal_text = """Evidence may be given in any suit or proceeding of the existence or non-existence of every fact in issue and of such other facts as are hereinafter declared to be relevant, and of no others.

Explanation.— This section shall not enable any person to give evidence of a fact which he is disentitled to prove by any provision of the law for the time being in force relating to civil procedure."""

# Many T5-style models expect an instruction prefix; try both:
inputs = "simplify: " + legal_text
out = simplifier(inputs, max_length=512, do_sample=False)   # deterministic output

print(out[0]["generated_text"])


Device set to use cpu
Both `max_new_tokens` (=256) and `max_length`(=512) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


This section shall not allow any person to give evidence of a fact that he is not entitled to prove by any provision of the law for the time being in force relating to civil procedure.


In [13]:
from transformers import pipeline

# Load a simplification fine-tuned model
model_name = "agentlans/flan-t5-small-simplifier"  # You can also try "mrm8488/t5-small-finetuned-text-simplification"

simplifier = pipeline(
    "text2text-generation",
    model=model_name,
    tokenizer=model_name,
    device=-1  # change to 0 if using GPU
)

legal_text = """Evidence may be given in any suit or proceeding of the existence or non-existence of every fact in issue and of such other facts as are hereinafter declared to be relevant, and of no others.

Explanation.— This section shall not enable any person to give evidence of a fact which he is disentitled to prove by any provision of the law for the time being in force relating to civil procedure."""

# Clear instruction for simplification
inputs = "Simplify the following legal text into clear, plain English while keeping all legal meaning:\n\n" + legal_text

# Generate simplified text
out = simplifier(
    inputs,
    max_length=600,       # allow longer outputs
    do_sample=False,      # deterministic output
    truncation=True
)

# Print the simplified version
print(out[0]["generated_text"])


Device set to use cpu
Both `max_new_tokens` (=256) and `max_length`(=600) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Simplify the following legal text into clear, plain English while keeping all legal meaning: Evidence may be given in any suit or proceeding of the existence or non-existence of every fact in issue and of such other facts as are hereinafter declared to be relevant, and of no others. Explanation.— This section shall not allow any person to give evidence of a fact which he is entitled to prove by any provision of the law for the time being in force relating to civil procedure.
